# Effect of Data Augmentation 

In each part we had normalized the data. 

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#=====================================================================================

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(22*22*64, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = (F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
def Accuracy(dataloader,model):     
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    for data in dataloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return 100 * correct / total



class CNNModel:
  def __init__(self, batch_size,epochs,lr,augf):
    self.batchSize = batch_size
    self.epochs = epochs
    self.learningRate = lr
    self.classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    self.trainloader = None
    self.valLoader = None
    self.AccTrain = []
    self.AccVal = []
    self.Epoch = []
    self.Model = None
    self.loss = {}
    self.err = {}
    self.lossT = []
    # self.err['train'] = []
    self.lossV = []
    # self.err['val'] = []
    self.best = None
    self.bestLossT = None
    self.bestLossV = None
    self.augFn = augf

    # self.xepoch

  def classWiseAcc(self, dataloader, model):
      correct_pred = {classname: 0 for classname in self.classes}
      total_pred = {classname: 0 for classname in self.classes}

      # again no gradients needed
      with torch.no_grad():
          for data in dataloader:
              images, labels = data
              outputs = model(images)
              _, predictions = torch.max(outputs, 1)
              # collect the correct predictions for each class
              for label, prediction in zip(labels, predictions):
                  if label == prediction:
                      correct_pred[self.classes[label]] += 1
                  total_pred[self.classes[label]] += 1


      # print accuracy for each class
      for classname, correct_count in correct_pred.items():
          accuracy = 100 * float(correct_count) / total_pred[classname]
          print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')


 
  def loadData(self):
    # Data transformations 
    no_transform = transforms.Compose(
        [transforms.ToTensor()
        ])
    normal_transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    flip_transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ,transforms.RandomHorizontalFlip(), transforms.RandomCrop(size=32)
        ])
    transform = no_transform
    if(self.augFn == 'F1'):
      transform  = no_transform
      print("Using no transformation ")
    elif(self.augFn == 'F2'):
      transform  = normal_transform
      print("Using normalize transformation ")
    elif(self.augFn == 'F3'):
      transform  = flip_transform
      print("Using flip transformation ")
    # Data Loading 

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=self.batchSize,
                                              shuffle=True, num_workers=2)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                          download=True, transform=normal_transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=self.batchSize,
                                            shuffle=False, num_workers=2)
    self.trainloader = trainloader
    self.valLoader = testloader
    print("Data Loaded")

  def valLoss(self,model,dataloader):
      running_loss = 0.0
      criterion = nn.CrossEntropyLoss()
      for i, data in enumerate(self.trainloader, 0):
          inputs, labels = data
          images = inputs.to(device)
          labels = labels.to(device)
          outputs = model(images)
          loss = criterion(outputs, labels)
          running_loss += loss.item()

      no_of_batches = 50000/self.batchSize
      return running_loss/ no_of_batches

  def train(self):
    batches = 50000/self.batchSize
    Train_Acc=[]
    Test_Acc=[]
    Train_Loss=[]
    Test_Loss = []
    Epochs=[]
    running_loss = 0.0
    running_corrects = 0.0
    minLossT = 100
    minLossV = 100

    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=self.learningRate)
    for epoch in range(self.epochs):  
      running_loss = 0.0
      for i, data in enumerate(self.trainloader, 0):
          inputs, labels = data
          images = inputs.to(device)
          labels = labels.to(device)
          # zero the parameter gradients
          optimizer.zero_grad()
          # Forward Pass
          outputs = model(images)
          loss = criterion(outputs, labels)
          # Backpropagation and optimizatuon

          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          # if i % 4000 == 3999:    # print every 2000 mini-batches
      print(f' Epoch: {epoch + 1} loss: {running_loss / batches:.3f}')
      train_loss = running_loss / batches
      if(train_loss<minLossT):
        minLossT = train_loss
        self.bestLossT = minLossT
      Train_Loss.append(train_loss)
      running_loss = 0.0
      with torch.no_grad():
        # testAcc = Accuracy(self.valLoader,model)
        # trainAcc = Accuracy(self.trainloader, model)
        # Train_Acc.append(trainAcc)
        # Test_Acc.append(testAcc)
        val_loss = self.valLoss(model,self.valLoader)
        if(val_loss<minLossV):
          minLossV = val_loss
          self.best = model
          self.bestLossV = minLossV
        Test_Loss.append(val_loss)
        Epochs.append(epoch+1)
        # print(f'Accuracy of the network on the 10000 test images: {testAcc} %')
        # print(f'Accuracy of the network on the train images: {trainAcc} %')
    self.Model = model
    self.AccVal = Test_Acc
    self.AccTrain = Train_Acc
    self.Epoch = Epochs
    self.lossT = Train_Loss
    self.lossV = Test_Loss 





In [ ]:
AugF = ['F1','F2','F3']
for f in AugF:
  print("================================================")
  M = CNNModel(4, 5, 0.005,f)
  M.loadData()
  M.train()
  bestModel = M.best
  acc = Accuracy(M.valLoader,bestModel)
  print("Accuracy for "+f+" is ",acc)


Using no transformation 
Files already downloaded and verified
Files already downloaded and verified
Data Loaded
 Epoch: 1 loss: 1.669
 Epoch: 2 loss: 1.243
 Epoch: 3 loss: 1.039
 Epoch: 4 loss: 0.892
 Epoch: 5 loss: 0.767
Accuracy for F1 is  47.83
Using normalize transformation 
Files already downloaded and verified
Files already downloaded and verified
Data Loaded
 Epoch: 1 loss: 1.492
 Epoch: 2 loss: 1.077
 Epoch: 3 loss: 0.882
 Epoch: 4 loss: 0.749
 Epoch: 5 loss: 0.634
Accuracy for F2 is  69.99
Using flip transformation 
Files already downloaded and verified
Files already downloaded and verified
Data Loaded
 Epoch: 1 loss: 1.509
 Epoch: 2 loss: 1.085
 Epoch: 3 loss: 0.907
 Epoch: 4 loss: 0.794
 Epoch: 5 loss: 0.715
Accuracy for F3 is  73.95
